In [1]:
import pandas as pd

### Read TU_TM1_**** and TC_TM1_****

In [2]:
tu_tm1 = pd.read_excel("X:/QSDOCS/PRD/DATA/Target/Weekly/RawData/Y26B New Reg R3.2 20250925_marcomm.xlsx",sheet_name="TU_TM1_7473")

In [3]:
tu_tm1

,Faculty,School,Cycle,Programme Name_TM1,INTAKE,New - Local,New - International,New - Progression
0,FSLM,TCI,C2,Advanced Diploma in Patisserie and Gastronomic...,202604,7,0,3
1,FSLM,TCI,C3,Advanced Diploma in Patisserie and Gastronomic...,202608,4,0,14
2,FSLM,SLAS,C1,American Degree Transfer Program (Business),202602,4,8,0
3,FSLM,SLAS,C2,American Degree Transfer Program (Business),202606,2,8,0
4,FSLM,SLAS,C3,American Degree Transfer Program (Business),202608,5,10,0
...,...,...,...,...,...,...,...,...
361,FBL,TBS,C1,Master of Management (ODL),202602,5,0,0
362,FBL,TBS,C2,Master of Management (ODL),202604,3,2,0
363,FBL,TBS,C2,Master of Management (ODL),202606,3,2,0
364,FBL,TBS,C3,Master of Management (ODL),202609,5,3,0


In [4]:
tu_tm1 =tu_tm1.drop(['Faculty','School'],axis=1)

In [5]:
#Add based on the current year | cycle
tu_tm1["Year"]='2026' 

In [6]:
tu_tm1=tu_tm1.rename(columns={'New - Local': 'Domestic', 'New - International': 'International', 'New - Progression': 'Progression'})

In [7]:
tu_tm1_melted_df = pd.melt(
    tu_tm1,
    id_vars=['Cycle', 'Programme Name_TM1', 'INTAKE', 'Year'],  # columns to keep
    value_vars=['Domestic', 'International', 'Progression'],    # columns to melt
    var_name='Market Segment',                                  # new column name for variable
    value_name='Student No'                                     # new column name for value
)


In [8]:
tu_tm1_melted_df = tu_tm1_melted_df.sort_values(by='Programme Name_TM1')

In [9]:
tu_tm1 = tu_tm1_melted_df

In [10]:
tc_tm1 = pd.read_excel("X:/QSDOCS/PRD/DATA/Target/Weekly/RawData/Y26B New Reg R3.2 20250925_marcomm.xlsx",sheet_name="TC_TM1_3092")

In [11]:
tc_tm1 =tc_tm1.drop(['Faculty','School'],axis=1)

In [12]:
tc_tm1=tc_tm1.rename(columns={'New - Local': 'Domestic', 'New - International': 'International', 'New - Progression': 'Progression'})

In [13]:
#Add based on the current year | cycle
tc_tm1['Year']='2026' 

In [14]:
tc_tm1 = pd.melt(
    tc_tm1,
    id_vars=['Cycle', 'Programme Name_TM1', 'INTAKE', 'Year'],  # columns to keep
    value_vars=['Domestic', 'International', 'Progression'],    # columns to melt
    var_name='Market Segment',                                  # new column name for variable
    value_name='Student No'                                     # new column name for value
)

In [15]:
# Combine the two DataFrames
combined_tc_tu = pd.concat([tc_tm1, tu_tm1], ignore_index=True) 

In [16]:
#Calculate the total number of students to make sure it is tally with the original file
total_students = combined_tc_tu['Student No'].sum()
total_students

np.int64(10565)

### Program Master File to map with TU TC budget target

In [17]:
PMF = pd.read_excel("X:/QSDOCS/PRD/DATA/Mapping/New_Program_Master.xlsx",sheet_name="PMF")

In [18]:
#PMF_1 = PMF[PMF['Excluded'] != 1]
PMF = PMF[PMF['Excluded'] != 1] 

In [19]:
PMF = PMF[['Init_Year',"Programme Code","Programme Name TM1"]]

In [20]:
# Sort the DataFrame by 'Programme Code' and 'Init_Year' in descending order - this is for checking purpose
df_sorted = PMF.sort_values(by=['Programme Name TM1', 'Init_Year'], ascending=[True, False])

In [21]:
# Remove duplicates based on 'Programme Code', keeping the first occurrence (which is the latest year due to sorting)
df_unique = df_sorted.drop_duplicates(subset='Programme Name TM1', keep='first')

In [22]:
PMF=df_unique

In [23]:
PMF.head(10)

,Init_Year,Programme Code,Programme Name TM1
39,2020,85003,"3+0 BACHELOR OF BUSINESS (MANAGEMENT), UNISA"
41,2020,85002,"3+0 BACHELOR OF COMMERCE (ACCOUNTING), UNISA"
0,2020,44405,ACCA (IRC)
5,2020,30042,Advanced Diploma in Patisserie and Gastronomic...
6,2018,5003,American Degree Transfer Program
7,2020,5004,American Degree Transfer Program (Business)
8,2022,5006,American Degree Transfer Program (Computer Sci...
10,2020,5005,American Degree Transfer Program (Engineering)
11,2020,5007,American Degree Transfer Program (Liberal Arts)
263,2025,94609,BACHELOR OF APPLIED HEALTH SCIENCES (HONOURS)


In [24]:
PMF['Programme Name TM1'] = PMF['Programme Name TM1'].str.strip()

C:\Users\112363\AppData\Local\Temp\ipykernel_15804\1175447385.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PMF['Programme Name TM1'] = PMF['Programme Name TM1'].str.strip()


In [25]:
#Standardize the columns to match with the target file
combined_tc_tu=combined_tc_tu.rename(columns={"Programme Name_TM1": "Programme Name TM1"})

In [26]:
combined_tc_tu["Programme Name TM1"]=combined_tc_tu["Programme Name TM1"].str.strip()

In [27]:
merged_df = pd.merge(combined_tc_tu, PMF,
                     how='left',
                     on="Programme Name TM1")

In [28]:
#For checking purposes
total_students = merged_df['Student No'].sum()

In [29]:
total_students

np.int64(10565)

In [30]:
merged_df = merged_df.sort_values(by='INTAKE')

### Export to excel

In [31]:
merged_df.to_excel("C:/Users/112363/Downloads/target.xlsx", index=False)